## 1. Load data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(str(dirname)+'/'+str(filename))

In [ ]:
df.head(2)

In [ ]:
df.isna().sum()

Drop duplicates

In [ ]:
df.drop_duplicates().reset_index(drop=True, inplace=True)

make CPM

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

In [ ]:
foo = lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 

In [ ]:
df['CPM'] = df.apply(foo, axis=1)

take CPM >=0 and drop total_revenue, measurable_impressions, revenue_share_percent

In [ ]:
df = df.loc[df['CPM']>=0]

In [ ]:
df.drop(['total_revenue','revenue_share_percent', 'measurable_impressions'], axis=1, inplace=True)

In [ ]:
df.head(2)

In [ ]:
df['CPM'].describe()

In [ ]:
df = df[df['CPM'] < df['CPM'].quantile(.95)] 

## 2.Train test split

In [ ]:
split_data = '2019-06-22'

test = df[df.date >= split_data]
train = df[df.date < split_data]

In [ ]:
X = train.drop(['date','CPM'], axis=1)
y = train['CPM']

In [ ]:
TEST_SIZE = 0.3
RAND_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state=RAND_STATE)

## 3. Model

### 3.1 Default models

In [ ]:
models = [Ridge, Lasso, RandomForestRegressor, GradientBoostingRegressor]
model_names = ['Ridge', 'Lasso', 'RandomForestRegressor', 'GradientBoostingRegressor']

def estimate_model(model_cls):
    model = model_cls(random_state=RAND_STATE)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = mean_squared_error(y_test, y_pred)
    return score

regression_scores = []
for model_cls in models:
    score = estimate_model(model_cls)
    regression_scores.append(score)

index = np.array(regression_scores).argmin()
answer = regression_scores[index]
print("best score: %s" % model_names[index])

### 3.2 Random Forest

In [ ]:
rfg = RandomForestRegressor()
rfg.fit(X_train, y_train)

y_pred_rfg = rfg.predict(X_test)

In [ ]:
print("MSE: ", (mean_squared_error(y_pred_rfg, y_test)))